In [2]:
#import packages

import os
import warnings

import earthpy as et
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import pandas as pd

#warnings.simplefilter('ignore')

In [3]:
# Download the water boundary data and store in geodataframe
wbd_url = (
    "https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/WBD/National/"
    "GDB/WBD_National_GDB.zip") 
wbd_dir = et.data.get_data(url=wbd_url)

wbd_path = os.path.join(wbd_dir, 'WBD_National_GDB.gdb')
wbd_hu2_gdf = gpd.read_file(wbd_path, layer='WBDHU2', from_disk=True)

#create test plot
# wbd_hu2_gdf.plot()

# Download, cache and unzip wildfire occurence data using earthpy
fire_url = (
"https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.6/"
"RDS-2013-0009.6_Data_Format2_GDB.zip")

fire_dir = et.data.get_data(url=fire_url)
fire_dir

#Download fire path table from database
import pyogrio
fire_path = os.path.join(fire_dir, 'Data', 'FPA_FOD_20221014.gdb')

if not 'fire_gdf' in globals():
    print('fire_gdf does not exist. Loading...')
    fire_gdf = pyogrio.read_dataframe(fire_path, layer='Fires')

# Print first five columns of geodataframe for testing
#fire_gdf.head()

# Create subsetted geodataframe with desired columns
fire_clean_gdf = (
    fire_gdf
    [['FOD_ID','DISCOVERY_DATE','FIRE_SIZE', 'geometry']]
    .set_index('FOD_ID')
)


KeyboardInterrupt: 

In [2]:
# Download the water boundary data and store in geodataframe
wbd_url = (
    "https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/WBD/National/"
    "GDB/WBD_National_GDB.zip") 
wbd_dir = et.data.get_data(url=wbd_url)

wbd_path = os.path.join(wbd_dir, 'WBD_National_GDB.gdb')
wbd_hu2_gdf = gpd.read_file(wbd_path, layer='WBDHU2', from_disk=True)

#create test plot
# wbd_hu2_gdf.plot()

# Download, cache and unzip wildfire occurence data using earthpy
fire_url = (
"https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.6/"
"RDS-2013-0009.6_Data_Format2_GDB.zip")

fire_dir = et.data.get_data(url=fire_url)
fire_dir

#Download fire path table from database
import pyogrio
fire_path = os.path.join(fire_dir, 'Data', 'FPA_FOD_20221014.gdb')

if not 'fire_gdf' in globals():
    print('fire_gdf does not exist. Loading...')
    fire_gdf = pyogrio.read_dataframe(fire_path, layer='Fires')

# Print first five columns of geodataframe for testing
#fire_gdf.head()

# Create subsetted geodataframe with desired columns
fire_clean_gdf = (
    fire_gdf
    [['FOD_ID','DISCOVERY_DATE','FIRE_SIZE', 'geometry']]
    .set_index('FOD_ID')
)

# Changing the CRS 
fire_clean_gdf.DISCOVERY_DATE = pd.to_datetime(fire_clean_gdf.DISCOVERY_DATE)
fire_clean_gdf = fire_clean_gdf.to_crs(wbd_hu2_gdf.crs)

first_year = min(fire_clean_gdf.DISCOVERY_DATE.dt.year)
last_year = max(fire_clean_gdf.DISCOVERY_DATE.dt.year)
step = 1
year_list = list(range(first_year, last_year +  1,step))

fire_clean_jan_gdf = fire_clean_gdf[pd.to_datetime(fire_clean_gdf['DISCOVERY_DATE']).dt.month == 1]
fire_clean_jan_yr_gdf =[]

for year in year_list:
    filtered_df = fire_clean_jan_gdf[pd.to_datetime(fire_clean_jan_gdf['DISCOVERY_DATE']).dt.year == year]
    fire_clean_jan_yr_gdf.append(filtered_df)





: 

In [ ]:
# Create geodataframe with annual # of fires and max fire size by watershed
fire_region_jan_gdf = (
    wbd_hu2_gdf
    .reset_index()
    [['name', 'geometry']]
    .sjoin(fire_clean_jan_gdf, how='inner', predicate='intersects')
)

fire_region_jan_gdf = fire_region_jan_gdf.reset_index()

fire_region_jan_gdf = (
    fire_region_jan_gdf
    .reset_index()
    .groupby(['name', fire_region_jan_gdf.DISCOVERY_DATE.dt.year])
    .agg(
        max_fire_size=('FIRE_SIZE', 'max'), 
        num_fires=('index_right', 'count'))
)

fire_region_jan_gdf = fire_region_jan_gdf.reset_index()
region_names = fire_region_jan_gdf.name.unique()

# Convert the 'DISCOVERY_DATE' column to a datetime format
fire_region_jan_gdf['DISCOVERY_DATE'] = pd.to_datetime(fire_region_jan_gdf['DISCOVERY_DATE'], format='%Y')

In [ ]:
# Filter the GeoDataFrame for the years 1992 to 2002
filtered_9202_gdf = fire_region_jan_gdf[(fire_region_jan_gdf['DISCOVERY_DATE'].dt.year >= 1992) & (fire_region_jan_gdf['DISCOVERY_DATE'].dt.year <= 2002)]

# Compute average annual January fires from 1992-2002 in each watershed
fire_count_jan_9202_gdf = (
    filtered_9202_gdf
    .reset_index()
    [['name', 'num_fires']]
    .groupby('name')
    .sum()/10
)

# Filter the GeoDataFrame for the years 2010 to 2020
filtered_1020_gdf = fire_region_jan_gdf[(fire_region_jan_gdf['DISCOVERY_DATE'].dt.year >= 2010) & (fire_region_jan_gdf['DISCOVERY_DATE'].dt.year <= 2020)]

# Compute average annual January fires from 2010-2020 in each watershed
fire_count_jan_1020_gdf = (
    filtered_1020_gdf
    .reset_index()
    [['name', 'num_fires']]
    .groupby('name')
    .sum()/10
)
fire_count_jan_1020_gdf = fire_count_jan_1020_gdf.reset_index()
fire_count_jan_1020_clean_gdf = fire_count_jan_1020_gdf[fire_count_jan_1020_gdf.name != 'Caribbean Region']
fire_count_jan_1020_clean_gdf = fire_count_jan_1020_clean_gdf.dropna(subset=['num_fires'])
fire_count_jan_1020_clean_gdf =  fire_count_jan_1020_clean_gdf.reset_index()

fire_count_jan_9202_gdf = fire_count_jan_9202_gdf.reset_index()
fire_count_jan_9202_gdf_copy = fire_count_jan_9202_gdf.copy()
fire_count_jan_9202_gdf_copy = fire_count_jan_9202_gdf_copy.reset_index()

fire_count_jan_1020_clean_gdf, fire_count_jan_9202_gdf = fire_count_jan_1020_clean_gdf.reindex(fire_count_jan_9202_gdf.index), fire_count_jan_9202_gdf

# Compute change in average annual January fires from 1992-2002 to 2010-2020 in each watershed
fire_change_jan_gdf = fire_count_jan_9202_gdf_copy  # Make a copy of one of the DataFrames to preserve its structure
fire_change_jan_gdf['fire_change'] = fire_count_jan_1020_clean_gdf['num_fires'] - fire_count_jan_9202_gdf['num_fires']

# set index for both dataframes to 'name' if it isn't already 'name'
if 'name' not in wbd_hu2_gdf.index.names:
    wbd_hu2_gdf.reset_index()
    wbd_hu2_gdf.set_index('name', inplace=True)
if 'name' not in fire_change_jan_gdf.index.names:
    wbd_hu2_gdf.reset_index()
    fire_change_jan_gdf.set_index('name', inplace=True)

# merge watershed geometry and january fire change datasets into dataframe
fire_change_watershd_jan_gdf = pd.merge(
    wbd_hu2_gdf[['geometry']],
    fire_change_jan_gdf[['fire_change','num_fires']], 
    left_index=True, 
    right_index=True)

## Winter wildfire in the United States

![Damage from the Marshall Fire covered in a fresh blanket of snow, Louisville, CO, January 2022](https://s.hdnux.com/photos/01/23/35/35/21878601/3/1200x0.jpg)

>Image Source: [San Francisco Chronicle](https://www.sfchronicle.com/california-wildfires/article/Could-Colorado-s-extreme-winter-wildfires-16744142.php#photo-21878608)

Wildfire is becoming more frequent and widespread across the United States due to climate change. Fire seasons are also growing longer, with destructive fires occurring earlier and later in the year. Colorado's Marshall Fire, which burnt more homes than any other fire in state history and caused two deaths, ignited on December 30th, well outside of the historical Colorado Fire Season.

## Fires in January are increasing

January wildfires are most common in the Southeast, where relatively warm temperatures and a lack of snowfall allow for fire. But as winters grow warmer, the pattern of winter wildfires is changing. 

In [ ]:
fire_change_watershd_jan_gdf.geometry = fire_change_watershd_jan_gdf.geometry.simplify(tolerance=.1)

poly_plot = (
    gv.Polygons(
        fire_change_watershd_jan_gdf
        .drop(['Alaska Region','Hawaii Region'], axis='rows')
            .reset_index()
            .dropna()
            [['num_fires', 'geometry']]
    )
    .opts(
        width=1000, height=600,
        colorbar=True, color='Number of January Fires',
        cmap='kr', clim=(0,300),line_color='white',
        xaxis='bare', yaxis='bare', tools=['hover'],
        title='Mean Annual January Fires, 1992-2002'
    )
)

gv.tile_sources.EsriTerrain * poly_plot

<span style="font-size: 130%;">Figure 1.</span> Mean annual fires in January during the period 1992-2002 by major watershed boundaries. Fires are more common in the Southeastern and South Central states, with northern areas seeing very few fires in January. 

> Data Sources:
> - Watershed Boundaries: [United States Geological Survey Water Boundary Dataset](https://www.usgs.gov/national-hydrography/watershed-boundary-dataset)
> - Wildfire Occurrence: Short, Karen C. 2022. Spatial wildfire occurrence data for the United States, 1992-2020 FPA_FOD_20221014. 6th Edition. Fort Collins, CO: Forest Service Research Data Archive. [https://doi.org/10.2737/RDS-2013-0009.6](https://doi.org/10.2737/RDS-2013-0009.6)

In [ ]:
poly_plot = (
    gv.Polygons(
        fire_change_watershd_jan_gdf
        .drop(['Alaska Region','Hawaii Region'], axis='rows')
            .reset_index()
            .dropna()
            [['fire_change', 'geometry']]
    )
    .opts(
        width=1000, height=600,
        colorbar=True, color='Change in January Fires',
        cmap='RdBu_r', clim=(-300,300), line_color='black',
        xaxis='bare', yaxis='bare', tools=['hover'],
        title='Change in mean annual January fires, 1992-2002 to 2010-2020'
    )
)

gv.tile_sources.EsriTerrain * poly_plot

<span style="font-size: 130%;">Figure 2.</span> Change in mean annual January fires. January fires are becoming less frequent in the Southeastern United States and more common in watersheds that span the Plains states and eastern Rocky Mountains, the desert Southwest and California.

## No snow, more fire?
The Western United States is seeing increasingly frequent winters with little or no snow on the ground, and these conditions are expected to become more persistent with further warming of the atmosphere. Snow-free winters will continue to have an impact on wildfire timing, frequency and severity, potentially intensifying wildfire activity throughout the year--including during the winter.

In [ ]:
%%capture
%%bash
jupyter nbconvert <jan_fires>.ipynb --to html --no-input